In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('elspeth_data.csv')

In [3]:
df.head()

,Unnamed: 0,person,date_of_report,ref,name_of_deceased,coroner_name,coroner_area,category,filename,text,url
0,0,malyun-karama-2020-0162_redacted,Date of report: 21 August 2020,2020-0162,Malyun Karama,NaN,NaN,Category: Hospital death (Clinical procedure a...,Malyun-Karama-2020-0162_Redacted.pdf,\n\n \n\n \n1 \n\n \n2 \n\n \n3 \n\n \n4 \n\n...,https://www.judiciary.uk/prevention-of-future-...
1,1,theresa-robertson-2020-0158_redacted,Date of report: 6 August 2020,2020-0158,Theresa Robertson,NaN,NaN,"Category: Alcohol, drug and medication related...",Theresa-Robertson-2020-0158_Redacted.pdf,Regulation 28: Prevention of Future Deaths re...,https://www.judiciary.uk/prevention-of-future-...
2,2,george-townsend-2020-0157_redacted,Date of report: 4 June 2020,2020-0157,George Townsend,NaN,NaN,"Category: Community healthcare related deaths,...",George-Townsend-2020-0157_Redacted.pdf,REGULATION 28: REPORT TO PREVENT FUTURE DEATH...,https://www.judiciary.uk/prevention-of-future-...
3,3,jerrelle-mckenzie-2020-0144_redacted,Date of report: 17 July 2020,2020-0144,Jerrelle McKenzie,NaN,NaN,"Category: Alcohol, drug and medication related...",Jerrelle-McKenzie-2020-0144_Redacted.pdf,47812-2019 \n\nSenior Coroner - Emma Whitting ...,https://www.judiciary.uk/prevention-of-future-...
4,4,joan-williams-2020-0128_redacted,Date of report: 16 June 2020,2020-0128,Joan Williams,NaN,NaN,Category: Road (Highways Safety) related deaths,Joan-Williams-2020-0128_Redacted.pdf,48060-2019\n\nSenior Coroner - Emma Whitting\n...,https://www.judiciary.uk/prevention-of-future-...


In [5]:
len(df)

11976

In [6]:
df.name_of_deceased.nunique()

5374

In [7]:
df.name_of_deceased.unique()

array(['Malyun Karama', 'Theresa Robertson', 'George Townsend', ...,
       'Susan Williams', 'Keith Weston', 'Barbara Mitchell'], dtype=object)

In [63]:
df2 = pd.DataFrame(df.name_of_deceased.unique()).reset_index()

In [64]:
df2.columns = ['jam','name']

In [23]:
df2['name'].to_csv('names_for_elspeth.csv', index=False)

In [ ]:
# ok but do we need to klean the kolumn of names karl?

In [26]:
df[df.name_of_deceased.isnull()] # are there any nulls?

,Unnamed: 0,person,date_of_report,ref,name_of_deceased,coroner_name,coroner_area,category,filename,text,url
122,122,marsden-2014-0373,Date of report: 14 August 2014,2014-0373,NaN,NaN,NaN,Category: Hospital (clinical procedures and me...,Marsden-2014-0373.pdf,REGULATION 28: REPORT TO PREVENT FUTURE DEATHS...,NaN
172,172,2015-0223-response-by-oxfordshire-county-council,NaN,2015-0223,NaN,NaN,NaN,NaN,2015-0223-Response-by-Oxfordshire-County-Counc...,OXFORDSHIRE\nCOUNTY COUNCIL\n\nDate: 16 April ...,https://www.judiciary.uk/prevention-of-future-...
260,260,avon-fire-pfd-response,NaN,https://www.judiciary.uk/prevention-of-future-...,NaN,NaN,NaN,NaN,Avon-Fire-PFD-response.pdf,\n\nTuesday 29 March 2016\n\nDr Harrowing\nC...,NaN
261,261,d-mccorkle-response,NaN,https://www.judiciary.uk/prevention-of-future-...,NaN,NaN,NaN,NaN,D-McCorkle-Response.pdf,\n\nga NOV 2016\n\n \n\nLewisham\nClinical ...,NaN
262,262,d-mostari-response,NaN,https://www.judiciary.uk/prevention-of-future-...,NaN,NaN,NaN,NaN,D-Mostari-Response.pdf,\n\nGh\n\nKempston Road\n\nBedford\n\nMK42 9D...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11599,11599,wilfred-fitchett-jevon-hirst-hugo-morris-and-h...,NaN,https://www.judiciary.uk/prevention-of-future-...,NaN,NaN,NaN,NaN,Wilfred-Fitchett-Jevon-Hirst-Hugo-Morris-and-H...,Kate Robertson\nSenior Coroner for North West ...,NaN
11662,11662,joseph-abbess-and-sunnah-khan-prevention-of-fu...,NaN,https://www.judiciary.uk/prevention-of-future-...,NaN,NaN,NaN,NaN,Joseph-Abbess-and-Sunnah-Khan-Prevention-of-Fu...,REGULATION 28: REPORT TO PREVENT FUTURE DEATH...,NaN
11663,11663,2024-0538-response-from-department-for-education,NaN,2024-0538,NaN,NaN,NaN,NaN,2024-0538-Response-from-Department-for-Educati...,\n\n \n\nMinister for School Standards \n\nDe...,NaN
11890,11890,2024-0158-response-from-west-yorkshire-icb,NaN,2024-0158,NaN,NaN,NaN,NaN,2024-0158-Response-from-West-Yorkshire-ICB.pdf,\n\n \n\n \n\nWhite Rose House \nWest Para...,NaN


In [37]:
df[df.person.str.contains('fitchett')] # maybe doesn't matter

,Unnamed: 0,person,date_of_report,ref,name_of_deceased,coroner_name,coroner_area,category,filename,text,url
11598,11598,wilfred-fitchett-jevon-hirst-hugo-morris-and-h...,Date of report: 17/10/2024,2024-0560,"Wilfred Fitchett, Jevon Hirst, Hugo Morris and...",Coroners name: Kate Robertson,Coroners Area: North West Wales,Category: Road (Highways Safety) related death...,Wilfred-Fitchett-Jevon-Hirst-Hugo-Morris-and-H...,Kate Robertson\nSenior Coroner for North West ...,https://www.judiciary.uk/prevention-of-future-...
11599,11599,wilfred-fitchett-jevon-hirst-hugo-morris-and-h...,NaN,https://www.judiciary.uk/prevention-of-future-...,NaN,NaN,NaN,NaN,Wilfred-Fitchett-Jevon-Hirst-Hugo-Morris-and-H...,Kate Robertson\nSenior Coroner for North West ...,NaN


In [38]:
df2.name.str.contains('Wilfred').value_counts()

False    5370
True        4
Name: name, dtype: int64

In [41]:
df2[df2.name.str.contains('Fitchett', na=False)] # doesn't matter but highlights another problem

,jam,name
5346,5346,"Wilfred Fitchett, Jevon Hirst, Hugo Morris and..."


In [70]:
# handling pfds where multiple people are named, lets unstack them and match to ethnicity and decide what to do later
# so we want a new row for each name where there are multiple name
# lets try splitting by ','

df2['name'] = df2['name'].str.split(',') # Split by comma

df2 = df2.explode('name') # Explode to get one name per row

df2['name'] = df2['name'].str.strip()# Strip whitespace


In [71]:
df2[df2.name.str.contains('Fitchett', na=False)]

,jam,name,n_of_words
5346,5346,Wilfred Fitchett,9


In [72]:
# progress. most names should be firstname secondname.that's two words. are any names not?

df2.name.astype(str).apply(lambda x: len(x.split())).value_counts() # yup, only 5193 of our names are composed of two words

2     5193
3      122
5       53
4       24
7       10
1        5
6        3
26       2
24       2
8        1
28       1
32       1
19       1
29       1
11       1
22       1
30       1
61       1
Name: name, dtype: int64

In [73]:
df2['n_of_words'] = df2.name.astype(str).apply(lambda x: len(x.split()))

In [74]:
df2[df2.n_of_words > 2] # cool, sometimes we list multiple people using a ',' and sometimes an 'and'

,jam,name,n_of_words
101,101,Yvonne Davies and Andrew Davies,5
137,137,Grace Mary Bates,3
139,139,John Gwynfryn Morris,3
140,140,Elsie May Treece,3
142,142,Isabella Hope Hill,3
...,...,...,...
5281,5281,Oscar Barker and Yousef Al-Kharboush,5
5312,5312,Janet Brown Townend,3
5315,5315,George Kyriacos Petrou,3
5318,5318,Sebastian ‘Benji’ Oliver,3


In [75]:
df2['name'] = df2['name'].str.split(' and ') # Split by and
df2 = df2.explode('name') # Explode to get one name per row
df2['name'] = df2['name'].str.strip()# Strip whitespace


In [76]:
df2[df2.n_of_words > 2] # wtf. how is yvonne davies 5 words, oh it's cos we failed to update our count

,jam,name,n_of_words
101,101,Yvonne Davies,5
101,101,Andrew Davies,5
137,137,Grace Mary Bates,3
139,139,John Gwynfryn Morris,3
140,140,Elsie May Treece,3
...,...,...,...
5312,5312,Janet Brown Townend,3
5315,5315,George Kyriacos Petrou,3
5318,5318,Sebastian ‘Benji’ Oliver,3
5346,5346,Hugo Morris,5


In [77]:
df2['n_of_words'] = df2.name.astype(str).apply(lambda x: len(x.split()))
df2[df2.n_of_words > 2]  # cool, sometimes we list multiple people using a ',' and sometimes an '&'

,jam,name,n_of_words
137,137,Grace Mary Bates,3
139,139,John Gwynfryn Morris,3
140,140,Elsie May Treece,3
142,142,Isabella Hope Hill,3
143,143,Philippine de Gerin-Ricard,3
...,...,...,...
5241,5241,Mnayea Al Basman,3
5266,5266,Muhammad & Naemat Esmael,4
5312,5312,Janet Brown Townend,3
5315,5315,George Kyriacos Petrou,3


In [81]:
df2['name'] = df2['name'].str.split(' & ') # Split by &
df2 = df2.explode('name') # Explode to get one name per row
df2['name'] = df2['name'].str.strip()# Strip whitespace


In [82]:
df2['n_of_words'] = df2.name.astype(str).apply(lambda x: len(x.split()))
df2[df2.n_of_words > 2]  # looks alrite

,jam,name,n_of_words
137,137,Grace Mary Bates,3
139,139,John Gwynfryn Morris,3
140,140,Elsie May Treece,3
142,142,Isabella Hope Hill,3
143,143,Philippine de Gerin-Ricard,3
...,...,...,...
5232,5232,Luke Albiston O’Donnell,3
5241,5241,Mnayea Al Basman,3
5312,5312,Janet Brown Townend,3
5315,5315,George Kyriacos Petrou,3


In [84]:
df2[df2.n_of_words > 3] # ok 'Deceased names' can get in the bin &&  cool, sometimes we list multiple people using a ',' and sometimes an '&' and sometimes ;

,jam,name,n_of_words
288,288,Deceased names: Rebecca Dobson,4
432,432,Edna Elsie Mary Eden,4
434,434,Billy Paul Thomas Salton,4
576,576,Deceased names: Richard Dyson,4
799,799,Deceased names: Alexandru Murgeanu,4
830,830,Deceased names: Dane Chinnery,4
846,846,Deceased names: Samantha Gould,4
982,982,Garrett Joseph Franklin Elsey,4
1091,1091,Deceased names: Zainab Hashim,4
1429,1429,Marco Lima De Araujo,4


In [88]:
df2.name = df2.name.str.replace('Deceased names:','')

In [85]:
df2['name'] = df2['name'].str.split(';') # Split by ;
df2 = df2.explode('name') # Explode to get one name per row
df2['name'] = df2['name'].str.strip()# Strip whitespace


In [89]:
df2['n_of_words'] = df2.name.astype(str).apply(lambda x: len(x.split()))
df2[df2.n_of_words > 3] # I don't know why the coroner pollues some of these, can lose

,jam,name,n_of_words
432,432,Edna Elsie Mary Eden,4
434,434,Billy Paul Thomas Salton,4
982,982,Garrett Joseph Franklin Elsey,4
1429,1429,Marco Lima De Araujo,4
1659,1659,Sadie Ann Jane McGrady,4
1757,1757,Borough Market Terror Attack,4
2031,2031,Peter Patrick Adrian Barnes,4
2206,2206,Yvonne Sydney Annie Perry,4
2526,2526,Damion Stanley Joseph Henson,4
3087,3087,Lucy Hannah Rose Bailey,4


In [92]:
df2.name = df2.name.str.replace(r'Coroner name:.*', '', regex=True)

In [98]:
df2['n_of_words'] = df2.name.astype(str).apply(lambda x: len(x.split()))
df2[df2.n_of_words > 3] # I don't know why the coroner pollues some of these, can lose

,jam,name,n_of_words
432,432,Edna Elsie Mary Eden,4
434,434,Billy Paul Thomas Salton,4
982,982,Garrett Joseph Franklin Elsey,4
1429,1429,Marco Lima De Araujo,4
1659,1659,Sadie Ann Jane McGrady,4
1757,1757,Borough Market Terror Attack,4
2031,2031,Peter Patrick Adrian Barnes,4
2206,2206,Yvonne Sydney Annie Perry,4
2526,2526,Damion Stanley Joseph Henson,4
3087,3087,Lucy Hannah Rose Bailey,4


In [99]:
df2['n_of_words'] = df2.name.astype(str).apply(lambda x: len(x.split()))

In [101]:
df2.n_of_words.value_counts() # how we doing.. how do 10 people only have 1 name?

2     5349
3      126
4       16
1       10
12       3
7        2
13       2
15       1
11       1
18       1
14       1
17       1
5        1
Name: n_of_words, dtype: int64

In [102]:
df2[df2.n_of_words == 1] # *sigh*

,jam,name,n_of_words
122,122,NaN,1
2980,2980,REDACTED,1
3495,3495,[REDACTED],1
3518,3518,KennethDaly,1
4245,4245,Emma,1
4245,4245,Ellette,1
4868,4868,Portsmouth,1
4993,4993,Bryan,1
5233,5233,Champagauri,1
5266,5266,Muhammad,1


In [109]:
df2[df2.n_of_words == 4] 

,jam,name,n_of_words
432,432,Edna Elsie Mary Eden,4
434,434,Billy Paul Thomas Salton,4
982,982,Garrett Joseph Franklin Elsey,4
1429,1429,Marco Lima De Araujo,4
1659,1659,Sadie Ann Jane McGrady,4
1757,1757,Borough Market Terror Attack,4
2031,2031,Peter Patrick Adrian Barnes,4
2206,2206,Yvonne Sydney Annie Perry,4
2526,2526,Damion Stanley Joseph Henson,4
3087,3087,Lucy Hannah Rose Bailey,4


In [110]:
# bored of data cleaning
df2.to_csv('somewhat_cleaner_datas_for_elspeth.csv', index=False)

In [118]:
# might be helpful for you to understand / see how pandas is representing the datas
df.head()

,Unnamed: 0,person,date_of_report,ref,name_of_deceased,coroner_name,coroner_area,category,filename,text,url
0,0,malyun-karama-2020-0162_redacted,Date of report: 21 August 2020,2020-0162,Malyun Karama,NaN,NaN,Category: Hospital death (Clinical procedure a...,Malyun-Karama-2020-0162_Redacted.pdf,\n\n \n\n \n1 \n\n \n2 \n\n \n3 \n\n \n4 \n\n...,https://www.judiciary.uk/prevention-of-future-...
1,1,theresa-robertson-2020-0158_redacted,Date of report: 6 August 2020,2020-0158,Theresa Robertson,NaN,NaN,"Category: Alcohol, drug and medication related...",Theresa-Robertson-2020-0158_Redacted.pdf,Regulation 28: Prevention of Future Deaths re...,https://www.judiciary.uk/prevention-of-future-...
2,2,george-townsend-2020-0157_redacted,Date of report: 4 June 2020,2020-0157,George Townsend,NaN,NaN,"Category: Community healthcare related deaths,...",George-Townsend-2020-0157_Redacted.pdf,REGULATION 28: REPORT TO PREVENT FUTURE DEATH...,https://www.judiciary.uk/prevention-of-future-...
3,3,jerrelle-mckenzie-2020-0144_redacted,Date of report: 17 July 2020,2020-0144,Jerrelle McKenzie,NaN,NaN,"Category: Alcohol, drug and medication related...",Jerrelle-McKenzie-2020-0144_Redacted.pdf,47812-2019 \n\nSenior Coroner - Emma Whitting ...,https://www.judiciary.uk/prevention-of-future-...
4,4,joan-williams-2020-0128_redacted,Date of report: 16 June 2020,2020-0128,Joan Williams,NaN,NaN,Category: Road (Highways Safety) related deaths,Joan-Williams-2020-0128_Redacted.pdf,48060-2019\n\nSenior Coroner - Emma Whitting\n...,https://www.judiciary.uk/prevention-of-future-...


In [119]:
df.text.values[0] #e.g if wanted to inspect value of text field in first row

' \n\n \n\n \n1 \n\n \n2 \n\n \n3 \n\n \n4 \n\nRegulation 28:  Prevention of Future Deaths report \n \nMalyun Habib KARAMA (died 20.02.20) \n \n\n \nTHIS REPORT IS BEING SENT TO: \n \n\n1.  Dr \n\n \n\nMedical Director \nRoyal Free Hospital \nPond Street \nLondon  NW3 2QG \n \n\n \nCORONER \n \nI am:   Coroner ME Hassell \n           Senior Coroner  \n           Inner North London \n           St Pancras Coroner’s Court \n           Camley Street \n           London  N1C 4PP \n \n \nCORONER’S LEGAL POWERS \n \nI make this report under the Coroners and Justice Act 2009,  \nparagraph 7, Schedule 5, and  \nThe Coroners (Investigations) Regulations 2013, \nregulations 28 and 29. \n \n \nINVESTIGATION and INQUEST \n \nOn 26 February 2020, I commenced an investigation into the death of \nMalyun Karama, aged 34 years. The investigation concluded at the end \nof the inquest on 12 August 2020. I made a narrative determination at \ninquest, which I attach. \n \n \nCIRCUMSTANCES OF THE DEATH \n \